In [1]:
import os
import shutil
os.environ['OMPI_MCA_rmaps_base_oversubscribe'] = '1'
n_threads = 96
os.environ['USE_PYGEOS'] = '0'
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'
os.environ['NUMEXPR_MAX_THREADS'] = f'{n_threads}'
os.environ['NUMEXPR_NUM_THREADS'] = f'{n_threads}'
os.environ['OMP_THREAD_LIMIT'] = f'{n_threads}'
os.environ["OMP_NUM_THREADS"] = f'{n_threads}'
os.environ["OPENBLAS_NUM_THREADS"] = f'{n_threads}' # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = f'{n_threads}' # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = f'{n_threads}' # export VECLIB_MAXIMUM_THREADS=4
import numpy as np
import skmap_bindings as sb
import time

In [2]:
n_class_1 = 2
n_class_2 = 3
n_pix = 5
lc_mat = np.empty((n_pix, 2), dtype=np.float32)
indicators_mat = np.empty((n_pix, n_class_1 + n_class_2), dtype=np.float32)

lc_mat[:,0] = np.random.randint(0, n_class_1, n_pix).astype(np.float32)
lc_mat[:,1] = np.random.randint(0, n_class_2, n_pix).astype(np.float32)
print(lc_mat)

# Extact indicators for each LC (the parallelization is per pixels chunk so it will be fast)
# Parameters: data_in [matrix<float>], n_threads [uint], data_out[matrix<float>], col_in_select [uint], col_out_select [vector<uint>], classes [vector<uint>]
sb.extractIndicators(lc_mat, n_threads, indicators_mat, 0, range(0, n_class_1), range(0, n_class_1))
sb.extractIndicators(lc_mat, n_threads, indicators_mat, 1, range(n_class_1, n_class_1+n_class_2), range(0, n_class_2))
print(indicators_mat)

[[1. 1.]
 [1. 1.]
 [1. 2.]
 [0. 2.]
 [1. 0.]]
[[0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1.]
 [0. 1. 1. 0. 0.]]


In [3]:
n_class_1 = 70
n_class_2 = 120
n_pix = 4000*4000

lc_mat = np.empty((n_pix, 2), dtype=np.float32)
indicators_mat = np.empty((n_pix, n_class_1 + n_class_2), dtype=np.float32)

lc_mat[:,0] = np.random.randint(0, n_class_1, n_pix).astype(np.float32)
lc_mat[:,1] = np.random.randint(0, n_class_2, n_pix).astype(np.float32)
lc_mat

array([[ 37.,  67.],
       [ 63.,  65.],
       [ 12., 110.],
       ...,
       [ 30., 103.],
       [ 38.,  14.],
       [ 65., 108.]], dtype=float32)

In [4]:
start = time.time()
sb.extractIndicators(lc_mat, n_threads, indicators_mat, 0, range(0, n_class_1), range(0, n_class_1))
sb.extractIndicators(lc_mat, n_threads, indicators_mat, 1, range(n_class_1, n_class_1+n_class_2), range(0, n_class_2))
print(f"Time on real case application: {(time.time() - start):.2f} s")

Time on real case application: 6.54 s
